In [40]:
import pandas as pd
import datetime
import json
import numpy as np
import jsonlines

In [41]:
d_parser = lambda d, h: pd.datetime.strptime(d, '%d-%m-%Y')
table_2021 = pd.read_html('https://www.nordpoolgroup.com/492498/globalassets/marketdata-excel-files/elspot-prices_2021_hourly_sek.xls', decimal=',', thousands='', header=2)
df_2021 = table_2021[0]
table_2022 = pd.read_html('https://www.nordpoolgroup.com/492498/globalassets/marketdata-excel-files/elspot-prices_2022_hourly_sek.xls', decimal=',', thousands='', header=2)
df_2022 = table_2022[0]

In [42]:
df = pd.concat([df_2021, df_2022])

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10297 entries, 0 to 1535
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10297 non-null  object 
 1   Hours       10297 non-null  object 
 2   SYS         10296 non-null  float64
 3   SE1         10296 non-null  float64
 4   SE2         10296 non-null  float64
 5   SE3         10296 non-null  float64
 6   SE4         10296 non-null  float64
 7   FI          10296 non-null  float64
 8   DK1         10296 non-null  float64
 9   DK2         10296 non-null  float64
 10  Oslo        10296 non-null  float64
 11  Kr.sand     10296 non-null  float64
 12  Bergen      10296 non-null  float64
 13  Molde       10296 non-null  float64
 14  Tr.heim     10296 non-null  float64
 15  TromsÃ¸     10296 non-null  float64
 16  EE          10296 non-null  float64
 17  LV          10296 non-null  float64
 18  LT          10296 non-null  float64
 19  AT          0 non-null    

In [44]:
df

,Unnamed: 0,Hours,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,...,Tr.heim,TromsÃ¸,EE,LV,LT,AT,BE,DE-LU,FR,NL
0,01-01-2021,00 - 01,248.64,250.75,250.75,250.75,250.75,250.75,511.25,511.25,...,238.89,187.44,250.75,250.75,250.75,NaN,NaN,NaN,NaN,NaN
1,01-01-2021,01 - 02,244.02,244.72,244.72,244.72,244.72,244.72,484.32,484.32,...,230.45,187.03,244.72,244.72,244.72,NaN,NaN,NaN,NaN,NaN
2,01-01-2021,02 - 03,240.20,241.00,241.00,241.00,241.00,241.00,449.04,449.04,...,237.99,186.63,241.00,241.00,241.00,NaN,NaN,NaN,NaN,NaN
3,01-01-2021,03 - 04,237.29,238.39,238.39,238.39,238.39,238.39,431.35,431.35,...,237.49,186.63,238.39,238.39,238.39,NaN,NaN,NaN,NaN,NaN
4,01-01-2021,04 - 05,237.79,238.49,238.49,238.49,238.49,238.49,405.93,405.93,...,238.29,186.83,238.49,238.49,238.49,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,05-03-2022,19 - 20,1830.51,152.65,152.65,2121.46,2427.30,1186.47,4415.53,4415.53,...,152.65,152.65,2427.30,2427.30,2427.30,NaN,NaN,NaN,NaN,NaN
1532,05-03-2022,20 - 21,1784.23,150.92,150.92,2044.11,2427.30,954.63,4315.20,4315.20,...,150.92,150.92,2427.30,2427.30,2427.30,NaN,NaN,NaN,NaN,NaN
1533,05-03-2022,21 - 22,1779.27,149.09,149.09,2000.74,2751.05,1293.70,4069.78,4069.78,...,149.09,149.09,2751.05,2751.05,2751.05,NaN,NaN,NaN,NaN,NaN
1534,05-03-2022,22 - 23,1732.02,143.80,143.80,1293.81,1761.57,1155.40,3987.57,3987.57,...,143.80,143.80,1761.57,1761.57,1761.57,NaN,NaN,NaN,NaN,NaN


In [45]:
# Döp om första kolumnen till Date
df.rename(columns={'Unnamed: 0':'Date'}, inplace=True )

# Omvandla kolumerna Date och Hours till en DateTime och gör den till index för dataframe
df['DateTime'] = df['Date'] + ' ' + df['Hours'].str.replace(r'\s-.*', '')
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%d-%m-%Y %H')
df.set_index('DateTime', inplace=True)

# Ta bort oanvända kolumner
df.drop(columns=['Date', 'Hours'], axis=1, inplace=True)

/tmp/ipykernel_122/544344638.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DateTime'] = df['Date'] + ' ' + df['Hours'].str.replace(r'\s-.*', '')


In [46]:
# Omvandla från MWh till kWh
df.loc[:,(df.dtypes=='float64').values] = df.loc[:,(df.dtypes=='float64').values].div(1000)

In [47]:
df.head()

,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,Oslo,Kr.sand,...,Tr.heim,TromsÃ¸,EE,LV,LT,AT,BE,DE-LU,FR,NL
DateTime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,0.24864,0.25075,0.25075,0.25075,0.25075,0.25075,0.51125,0.51125,0.25075,0.25075,...,0.23889,0.18744,0.25075,0.25075,0.25075,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,0.24402,0.24472,0.24472,0.24472,0.24472,0.24472,0.48432,0.48432,0.24472,0.24472,...,0.23045,0.18703,0.24472,0.24472,0.24472,NaN,NaN,NaN,NaN,NaN
2021-01-01 02:00:00,0.24020,0.24100,0.24100,0.24100,0.24100,0.24100,0.44904,0.44904,0.24100,0.24100,...,0.23799,0.18663,0.24100,0.24100,0.24100,NaN,NaN,NaN,NaN,NaN
2021-01-01 03:00:00,0.23729,0.23839,0.23839,0.23839,0.23839,0.23839,0.43135,0.43135,0.23839,0.23839,...,0.23749,0.18663,0.23839,0.23839,0.23839,NaN,NaN,NaN,NaN,NaN
2021-01-01 04:00:00,0.23779,0.23849,0.23849,0.23849,0.23849,0.23849,0.40593,0.40593,0.23849,0.23849,...,0.23829,0.18683,0.23849,0.23849,0.23849,NaN,NaN,NaN,NaN,NaN


In [48]:
df['SE3']

DateTime
2021-01-01 00:00:00    0.25075
2021-01-01 01:00:00    0.24472
2021-01-01 02:00:00    0.24100
2021-01-01 03:00:00    0.23839
2021-01-01 04:00:00    0.23849
                        ...   
2022-03-05 19:00:00    2.12146
2022-03-05 20:00:00    2.04411
2022-03-05 21:00:00    2.00074
2022-03-05 22:00:00    1.29381
2022-03-05 23:00:00    0.79982
Name: SE3, Length: 10297, dtype: float64

In [49]:
timeseries = df['SE3']
print(timeseries)

DateTime
2021-01-01 00:00:00    0.25075
2021-01-01 01:00:00    0.24472
2021-01-01 02:00:00    0.24100
2021-01-01 03:00:00    0.23839
2021-01-01 04:00:00    0.23849
                        ...   
2022-03-05 19:00:00    2.12146
2022-03-05 20:00:00    2.04411
2022-03-05 21:00:00    2.00074
2022-03-05 22:00:00    1.29381
2022-03-05 23:00:00    0.79982
Name: SE3, Length: 10297, dtype: float64


# "Hämta" dataframe med elpris

In [50]:
df_price = df.round(decimals=4)
df_price['SE3']

DateTime
2021-01-01 00:00:00    0.2507
2021-01-01 01:00:00    0.2447
2021-01-01 02:00:00    0.2410
2021-01-01 03:00:00    0.2384
2021-01-01 04:00:00    0.2385
                        ...  
2022-03-05 19:00:00    2.1215
2022-03-05 20:00:00    2.0441
2022-03-05 21:00:00    2.0007
2022-03-05 22:00:00    1.2938
2022-03-05 23:00:00    0.7998
Name: SE3, Length: 10297, dtype: float64

In [51]:
df_price['SE3'].to_json('./price.json', orient='table')

# Hämta dataframe med väderdata

In [52]:
df_weather = pd.read_pickle('./df_days_all.pkl').round(decimals=2).replace(0,0.1)

# Sätt samman elpris med väderdata

In [53]:
split_train_test = 0.3

start_date = max(df_price.first_valid_index(), df_weather.first_valid_index(), key=lambda x:x.value)
end_date = min(df_price.last_valid_index(), df_weather.last_valid_index(), key=lambda x:x.value) - pd.offsets.DateOffset(days=10)

start_date_training = start_date
end_date_training = min(df_price.last_valid_index(), df_weather.last_valid_index(), key=lambda x:x.value) - pd.offsets.DateOffset(days=int((end_date - start_date).days * split_train_test))

start_date_test = end_date_training + pd.offsets.DateOffset(hours=1)
end_date_test = end_date

print('Training')
print(start_date_training)
print(end_date_training)

print('\nTest')
print(start_date_test)
print(end_date_test)


Training
2021-12-21 00:00:00
2022-02-14 23:00:00

Test
2022-02-15 00:00:00
2022-02-23 23:00:00


In [54]:
delta = datetime.timedelta(days=1)

# Training data
with open('training.jsonl', 'w') as outfile:
    outfile.close()

while start_date_training <= end_date_training:
    start_date_training_w_offset = pd.Timestamp(start_date_training) + pd.offsets.DateOffset(days=8)

    df_target = df_price['SE3'].loc[start_date_training.strftime("%Y-%m-%d"):start_date_training_w_offset.strftime("%Y-%m-%d")]
    target = df_target.values.tolist()
    
    df_dynamic_feat = df_weather.filter(regex=start_date_training.strftime("%Y%m%d")).loc[start_date_training.strftime("%Y-%m-%d"):start_date_training_w_offset.strftime("%Y-%m-%d")].dropna().T

    if not df_dynamic_feat.empty and len(target) == len(df_dynamic_feat.T.values):
        dynamic_feat = df_dynamic_feat.values.tolist()
    else:
        dynamic_feat = []
        for _ in range(0,47):
            dynamic_feat_one_location = []
            for _ in range(0,len(target)):
                dynamic_feat_one_location.append(0)
            dynamic_feat.append(dynamic_feat_one_location)
    
    with open('training.jsonl', 'a') as outfile:
        json.dump({'start': str(start_date_training), 'target': target, 'dynamic_feat': dynamic_feat}, outfile)
        outfile.write('\n')
    
    start_date_training += delta
    
    
# Test data
with open('test.jsonl', 'w') as outfile:
    outfile.close()

while start_date_test <= end_date_test:
    start_date_test_w_offset = pd.Timestamp(start_date_test) + pd.offsets.DateOffset(days=8)

    df_target = df_price['SE3'].loc[start_date_test.strftime("%Y-%m-%d"):start_date_test_w_offset.strftime("%Y-%m-%d")]
    target = df_target.values.tolist()
    
    df_dynamic_feat = df_weather.filter(regex=start_date_test.strftime("%Y%m%d")).loc[start_date_test.strftime("%Y-%m-%d"):start_date_test_w_offset.strftime("%Y-%m-%d")].dropna().T

    if not df_dynamic_feat.empty and len(target) == len(df_dynamic_feat.T.values):
        dynamic_feat = df_dynamic_feat.values.tolist()
    else:
        dynamic_feat = []
        for _ in range(0,47):
            dynamic_feat_one_location = []
            for _ in range(0,len(target)):
                dynamic_feat_one_location.append(0)
            dynamic_feat.append(dynamic_feat_one_location)
    
    with open('test.jsonl', 'a') as outfile:
        json.dump({'start': str(start_date_test), 'target': target, 'dynamic_feat': dynamic_feat}, outfile)
        outfile.write('\n')
    
    start_date_test += delta
    
print('Done')

Done
